# Rootfinding
**Matthias Heinkenschloss**,
**Department of Computational and Applied Mathematics**, **Rice University**
**August 29, 2021**

Algorithms to compute an approximation of a root $x_*$ of the function $f: \mathbb{R} \rightarrow \mathbb{R}$.
 

In [4]:
# import packages
import numpy as np
import math as m
import matplotlib.pyplot as plt

In [5]:
# function
def f(x):
    fval = m.cos(x)*m.cosh(x)+1
    fderiv = -m.sin(x)*m.cosh(x)+m.cos(x)*m.sinh(x);
    return fval, fderiv

In [6]:
# Bisection Method
def bisection(f, a, b, tolx = 1.e-7, tolf = 1.e-7, maxit = 100 ): 
    '''Approximate solution of f(x)=0 on interval [a,b] by bisection method.
 
    Parameters
    ----------
    f : function
        Function for which we wan to to find a root.
        The call f(x)  should return the value or the value, derivative
    a,b : numbers
        Interval in which to search for a solution. bisection returns
        None if f(a)*f(b) >= 0 since a solution is not guaranteed.
    tolx : number (optional, default = 1.e-7)
        Algorithm stops when | a - b | < tolx
    tolf : number (optional, default = 1.e-7)
        Algorithm stops when | f(0.5*(a+b)) | < tolf
    maxit: integer (optional, default = 100)
        maximjm number of iterations
        

    Returns
    -------
    a,b : numbers
        f(a)*f(b) < 0; a root of f is located in (a, b)
    x : numbers
        0.5*(a+b) approximation of the root.
    ithist : np.array 
        iteration history; i-th row of ithist contains [it, a, b, c, fc]
    ifag : integer
        return flag
        iflag = 0   if |b-a| <= tolx, or   | f(x) | <= tolf
        iflag =  1  iteration terminated because maximum number of
                    iterations was reached. |b-a| > tolx, |f(x)| > tolf

    Matthias Heinkenschloss
    Department of Computational and Applied Mathematics
    Rice University
    August 29, 2021
    '''
    
    if b < a :
        t = a
        a = b
        b = t
    
    fa = f(a)
    fb = f(b)
    if( isinstance(fa, tuple) ):
        fa = fa[0]
    if( isinstance(fb, tuple) ):
        fb = fb[0]
        
    # check if a and b bound a root
    if fa*fb >= 0 :
        raise Exception(
         "The scalars a and b do not bound a root")
    
    it    = 0
    iflag = 0
    c = 0.5*(a+b)
    fc = f(c)
    if( isinstance(fc, tuple) ):
        fc = fc[0]
    ithist = np.array([it, a, b, c, fc])

    while( it < maxit and abs(b-a) > tolx and abs(fc) > tolf ):
        it = it+1;
        if fa*fc < 0 :
            b = c
            fb = fc
        else:
            a = c
            fa = fc

        c  = 0.5*(a+b)
        fc = f(c)
        if( isinstance(fc, tuple) ):
            fc = fc[0]
            
        ithist = np.vstack((ithist, np.array([it, a, b, c, fc]) ))

    # check why the bisection method truncated and set iflag
    if abs(b-a) > tolx and abs(fc) > tolf :
        # method truncated because maximum number of iterations reached
        iflag = 1
    
    return a, b, c, ithist, iflag
    
 

In [7]:
# run Bisection
a, b, x, ithist, iflag = bisection(f, 1.6, 2, tolf = 1.e-7)
print(f' Bisection method returned with iflag = {iflag:1d}')
print(' iter       a             b             c           f(c) ')
for i in np.arange(ithist.shape[0]):
    print(f' {ithist[i,0]:3.0f}  {ithist[i,1]:13.6e}  {ithist[i,2]:13.6e}  {ithist[i,3]:13.6e}  {ithist[i,4]:13.6e}')


 Bisection method returned with iflag = 0
 iter       a             b             c           f(c) 
   0   1.600000e+00   2.000000e+00   1.800000e+00   2.939756e-01
   1   1.800000e+00   2.000000e+00   1.900000e+00  -1.049169e-01
   2   1.800000e+00   1.900000e+00   1.850000e+00   1.019814e-01
   3   1.850000e+00   1.900000e+00   1.875000e+00   4.306174e-04
   4   1.875000e+00   1.900000e+00   1.887500e+00  -5.176422e-02
   5   1.875000e+00   1.887500e+00   1.881250e+00  -2.554760e-02
   6   1.875000e+00   1.881250e+00   1.878125e+00  -1.252876e-02
   7   1.875000e+00   1.878125e+00   1.876562e+00  -6.041648e-03
   8   1.875000e+00   1.876562e+00   1.875781e+00  -2.803660e-03
   9   1.875000e+00   1.875781e+00   1.875391e+00  -1.186058e-03
  10   1.875000e+00   1.875391e+00   1.875195e+00  -3.776043e-04
  11   1.875000e+00   1.875195e+00   1.875098e+00   2.653550e-05
  12   1.875098e+00   1.875195e+00   1.875146e+00  -1.755272e-04
  13   1.875098e+00   1.875146e+00   1.875122e+00  -7.4

In [8]:
# Regular Falsi
def regulafalsi(f, a, b, tolx = 1.e-7, tolf = 1.e-7, maxit = 100 ): 
    '''Approximate solution of f(x)=0 on interval [a,b] by Regula Falsi.
    
    Parameters
    ----------
    f : function
        Function for which we wan to to find a root.
        The call f(x)  should return the value or the value, derivative
    a,b : numbers
        Interval in which to search for a solution. bisection returns
        None if f(a)*f(b) >= 0 since a solution is not guaranteed.
    tolx : number (optional, default = 1.e-7)
        Algorithm stops when | a - b | < tolx
    tolf : number (optional, default = 1.e-7)
        Algorithm stops when | f(0.5*(a+b)) | < tolf
    maxit: integer (optional, default = 100)
        maximjm number of iterations
        

    Returns
    -------
    a,b : numbers
        f(a)*f(b) < 0; a root of f is located in (a, b)
    x : numbers
        0.5*(a+b) approximation of the root.
    ithist : np.array 
        iteration history; i-th row of ithist contains [it, a, b, c, fc]
    ifag : integer
        return flag
        iflag = 0   if |b-a| <= tolx, or   | f(x) | <= tolf
        iflag =  1  iteration terminated because maximum number of
                    iterations was reached. |b-a| > tolx, |f(x)| > tolf


    Matthias Heinkenschloss
    Department of Computational and Applied Mathematics
    Rice University
    August 29, 2021
    '''
    
    if b < a :
        t = a
        a = b
        b = t
    
    fa = f(a)
    fb = f(b)
    if( isinstance(fa, tuple) ):
        fa = fa[0]
    if( isinstance(fb, tuple) ):
        fb = fb[0]
        
    # check if a and b bound a root
    if fa*fb >= 0 :
        raise Exception(
         "The scalars a and b do not bound a root")
    
    it    = 0
    iflag = 0
    c = a - fa * (b-a)/(fb-fa)
    fc = f(c)
    if( isinstance(fc, tuple) ):
        fc = fc[0]
    ithist = np.array([it, a, b, c, fc])

    while( it < maxit and abs(b-a) > tolx and abs(fc) > tolf ):
        it = it+1;
        if fa*fc < 0 :
            b = c
            fb = fc
        else:
            a = c
            fa = fc

        c = a - fa * (b-a)/(fb-fa)
        fc = f(c)
        if( isinstance(fc, tuple) ):
            fc = fc[0]
            
        ithist = np.vstack((ithist, np.array([it, a, b, c, fc]) ))

    # check why the bisection method truncated and set iflag
    if abs(b-a) > tolx and abs(fc) > tolf :
        # bisection method truncated because maximum number of iterations reached
        iflag = 1
    
    return a, b, c, ithist, iflag
    
 

In [9]:
# run Regula Falsi
a, b, x, ithist, iflag = regulafalsi(f, 0, 2, tolf = 1.e-7)
print(f' Regula Falsi returned with iflag = {iflag:1d}')
print(' iter       a             b             c           f(c) ')
for i in np.arange(ithist.shape[0]):
    print(f' {ithist[i,0]:3.0f}  {ithist[i,1]:13.6e}  {ithist[i,2]:13.6e}  {ithist[i,3]:13.6e}  {ithist[i,4]:13.6e}')



 Regula Falsi returned with iflag = 0
 iter       a             b             c           f(c) 
   0   0.000000e+00   2.000000e+00   1.559074e+00   1.029099e+00
   1   1.559074e+00   2.000000e+00   1.843610e+00   1.273375e-01
   2   1.843610e+00   2.000000e+00   1.872348e+00   1.138280e-02
   3   1.872348e+00   2.000000e+00   1.874866e+00   9.849238e-04
   4   1.874866e+00   2.000000e+00   1.875084e+00   8.498041e-05
   5   1.875084e+00   2.000000e+00   1.875102e+00   7.330408e-06
   6   1.875102e+00   2.000000e+00   1.875104e+00   6.323074e-07
   7   1.875104e+00   2.000000e+00   1.875104e+00   5.454156e-08


In [10]:
# Newton's method
def newt1d(f, x, tolx = 1.e-7, tolf = 1.e-7, maxit = 100 ): 
    '''Approximate solution of f(x)=0 using Newton;t method.
    
    Parameters
    ----------
    f : function
        Function for which we wan to to find a root.
        The call f(x)  should return  value, derivative
    x : number
        Initial approximation of root
    tolx : number (optional, default = 1.e-7)
        Algorithm stops when | s | < tolx, where s = Newton step
    tolf : number (optional, default = 1.e-7)
        Algorithm stops when | f(x | < tolf
    maxit: integer (optional, default = 100)
        maximjm number of iterations
        

    Returns
    -------
    x : numbers
        approximation of the root.
    ithist : np.array 
        iteration history; i-th row of ithist contains [it, x, fx, s]
    ifag : integer
        return flag
        iflag = 0   if |b-a| <= tolx, or   | f(x) | <= tolf
        iflag =  1  iteration terminated because maximum number of
                    iterations was reached. |b-a| > tolx, |f(x)| > tolf


    Matthias Heinkenschloss
    Department of Computational and Applied Mathematics
    Rice University
    August 29, 2021
    ''' 
   
    it    = 0
    iflag = 0
    
    fx, fpx = f(x)
    s  = - fx/fpx
    ithist = np.array([it, x, fx, s])
    
    while( it < maxit and abs(s) > tolx and abs(fx) > tolf ):
        x = x+s
        fx, fpx = f(x)
        s  = - fx/fpx
        it = it+1;
        
            
        ithist = np.vstack((ithist, np.array([it, x, fx, s]) ))

    # check why the bisection method truncated and set iflag
    if abs(s) > tolx and abs(fx) > tolf :
        # bisection method truncated because maximum number of iterations reached
        iflag = 1
    
    return x, ithist, iflag
    
 

In [12]:
# run Newton's method
x, ithist, iflag = newt1d(f, 2.0, tolf = 1.e-7)
print(f" Newton's method returned with iflag = {iflag:1d}")
print(' iter        x           f(x)           step')
for i in np.arange(ithist.shape[0]):
    print(f' {ithist[i,0]:3.0f}  {ithist[i,1]:13.6e}  {ithist[i,2]:13.6e}  {ithist[i,3]:13.6e}')



 Newton's method returned with iflag = 0
 iter        x           f(x)           step
   0   2.000000e+00  -5.656258e-01  -1.147253e-01
   1   1.885275e+00  -4.240234e-02  -1.009542e-02
   2   1.875179e+00  -3.111450e-04  -7.518138e-05
   3   1.875104e+00  -1.717092e-08  -4.149436e-09
